A custom chatbot with RAG to question information about a game, and answer from critic reviews.

In [4]:
import chromadb

chroma_client = chromadb.HttpClient(host='localhost', port=8000)

In [3]:
from chromadb.utils import embedding_functions

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

/Users/michaelcheng/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Get a collection

In [5]:
collection = chroma_client.get_collection('test_cyberpunk2077')

In [6]:
collection.peek()

{'ids': ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17'],
 'embeddings': [[-0.019422098994255066,
   -0.041043274104595184,
   0.0038442942313849926,
   -0.10659322142601013,
   0.03450283780694008,
   0.05725393071770668,
   -0.05685386434197426,
   0.0372808538377285,
   -0.062301069498062134,
   0.04936378076672554,
   -0.020209528505802155,
   0.06039934232831001,
   0.02548854425549507,
   -0.061949651688337326,
   0.06725990772247314,
   -0.029446423053741455,
   0.06677216291427612,
   -0.048048969358205795,
   0.061355359852313995,
   -0.03707921877503395,
   -0.035855650901794434,
   -0.06116443872451782,
   0.01550999190658331,
   0.06183855980634689,
   -0.009271068498492241,
   0.02996378391981125,
   -0.04645361751317978,
   -0.02660229615867138,
   -0.045874517410993576,
   -0.035626694560050964,
   0.0037679304368793964,
   0.06300494074821472,
   -0.05192856118083,
   -0.059518806636333466,
   0.0019657560624182224,
   -0.09857206046581268,
   0.05589458346366

---

LLM RAG with chromadb, build a custom chatbot

In [7]:
from langchain_community.llms import Ollama

In [8]:
llm = Ollama(model="llama2")

In [9]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# specify the collection name (same as chromadb)
db = Chroma(collection_name="test_collection", client=chroma_client, embedding_function=embedding_function)

In [15]:
# write a system prompt

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain


system_prompt = "You are a helpful assistant. Answer all questions to the best of your ability. If you don't know the answer, just say \'\'\'I do not know the answer.\'\'\'."

prompt_base = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
])

chain = create_stuff_documents_chain(llm, prompt_base)

ValueError: Prompt must accept context as an input variable. Received prompt with input variables: ['messages']

In [12]:
# add message history
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

In [13]:
# provide a user prompt

user_prompt_01 = "Is this game support multiplayer?"

# add the user message to the history
history.add_user_message(user_prompt_01)

history.messages

[HumanMessage(content='Is this game support multiplayer?')]

In [14]:
response = chain.invoke({"messages": history.messages})

response

'Yes, the game supports multiplayer.'